# Introduction

In [1]:
from uarray import *

## Creating Arrays

### Scalar

We can add custom values as scalars:

In [2]:
a_scalar = Scalar(123)
print(a_scalar)
a_scalar

123


Scalar('123')

Now we can take the shape of a scalar:

In [3]:
a_scalar_shape = Shape(a_scalar)
a_scalar_shape

Shape(Scalar('123'))

We see that the shape is not computed.

It just contains a reference to the arrays it operates on:

In [4]:
a_scalar_shape.operands

[Scalar('123')]

It also has a shorter representation we can print based on the Mathematics of Arrays symbols:

In [5]:
print(a_scalar_shape)

ρ(123)


We call `replace` to keep replacing expressions until no more are matched:

In [6]:
replace(a_scalar_shape)

Vector()

### Vector

We see that the shape of a scalar is `Vector`. `Vector` is a special operation that takes any number of operands. Each operand should be a scalar. We can create them with the `vector` helper function:

In [7]:
??vector

Signature: vector(*values)
Docstring: <no docstring>
Source:   
def vector(*values):
    return Vector(*map(Scalar, values))
File:      ~/p/uarray/uarray/uarray.py
Type:      function


In [8]:
a_vector = vector(1, 2, 3)
print(a_vector)
a_vector

<1 2 3>


Vector(Scalar('1'), Scalar('2'), Scalar('3'))

We can see the shape of a vector:

In [9]:
a_vector_shape = replace(Shape(a_vector))
print(a_vector_shape)

<3>


Which is itself a vector. 

### Iota

Now let's do something more interesting and create a range of values:

In [10]:
a_range = Iota(Scalar(10))
print(a_range)

ι(10)


In [11]:
print(replace(Shape(a_range)))

<10>


This is a vector of length 10. Let's see that the 5th element is the integer 5.

In [12]:
replace(Index(vector(5), a_range))

Scalar('5')

Here we use a new command `Index` which takes in a vector of indices and the an array and returns a subarray.

# Paper Example

Now let's look at the example from the paper in `uarray-docs`.

In [13]:
A = AbstractWithDimension(Scalar(3), variable_name="A")
B = AbstractWithDimension(Scalar(3), variable_name="B")
r = InnerProduct(
    Index(vector(1, 0), A),
    Scalar(Add),
    Scalar(Multiply),
    Index(
        vector(2),
        OuterProduct(
            Index(vector(1, 0), A),
            Scalar(Multiply),
            Index(vector(0, 1), B)
        )
    )
)
print(r)

((<1 0> ψ A^3) +·× (<2> ψ ((<1 0> ψ A^3) ·× (<0 1> ψ B^3))))


In [14]:
replaced_r = replace(r)
print(replaced_r)

((<1 0> ψp A^3) +·× (((<1 0> ‡v (<1> ↑ <2>)) ψf A^3) ·× ((<0 1> ‡v (<1> ↓ <2>)) ψp B^3)))


In [15]:
rhs = replaced_r[-1]
print(rhs)

(((<1 0> ‡v (<1> ↑ <2>)) ψf A^3) ·× ((<0 1> ‡v (<1> ↓ <2>)) ψp B^3))


In [16]:
i = AbstractWithDimension(Scalar(0), variable_name="i")

rr_ = Index(Vector(i), rhs)
print(rr_)
rr = replace(rr_)
print(rr)

(<i^0> ψ (((<1 0> ‡v (<1> ↑ <2>)) ψf A^3) ·× ((<0 1> ‡v (<1> ↓ <2>)) ψp B^3)))
(((<0> ↑ <i^0>) ψf ((<1 0> ‡v (<1> ↑ <2>)) ψf A^3)) × ((<0> ↓ <i^0>) ψf ((<0 1> ‡v (<1> ↓ <2>)) ψp B^3)))


In [17]:
print(replace(Index(Vector(), rr)))

(((<1> ≡v ExplodeVector((<0> ψf ρ(ρ(ρ(ρ((((<0> ↑ <i^0>) ψf ((<1 0> ‡v (<1> ↑ <2>)) ψf A^3)) × ((<0> ↓ <i^0>) ψf ((<0 1> ‡v (<1> ↓ <2>)) ψp B^3)))))))), ρ(ρ(ρ((((<0> ↑ <i^0>) ψf ((<1 0> ‡v (<1> ↑ <2>)) ψf A^3)) × ((<0> ↓ <i^0>) ψf ((<0 1> ‡v (<1> ↓ <2>)) ψp B^3)))))))) and (ExplodeVector(τ(ρ(<>)), rav(ρ(<>))) ≡v ExplodeVector(τ(ρ(ρ((((<0> ↑ <i^0>) ψf ((<1 0> ‡v (<1> ↑ <2>)) ψf A^3)) × ((<0> ↓ <i^0>) ψf ((<0 1> ‡v (<1> ↓ <2>)) ψp B^3)))))), rav(ρ(ρ((((<0> ↑ <i^0>) ψf ((<1 0> ‡v (<1> ↑ <2>)) ψf A^3)) × ((<0> ↓ <i^0>) ψf ((<0 1> ‡v (<1> ↓ <2>)) ψp B^3))))))))) ? (<> ψf (((<0> ↑ <i^0>) ψf ((<1 0> ‡v (<1> ↑ <2>)) ψf A^3)) × ((<0> ↓ <i^0>) ψf ((<0 1> ‡v (<1> ↓ <2>)) ψp B^3)))) : (<> ψp (((<0> ↑ <i^0>) ψf ((<1 0> ‡v (<1> ↑ <2>)) ψf A^3)) × ((<0> ↓ <i^0>) ψf ((<0 1> ‡v (<1> ↓ <2>)) ψp B^3)))))
